# Ray RLlib Tutorial - Exercise Solutions

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

This notebook contains the solutions for all the exercises in the RLlib tutorial.

First, we have to setup everything needed from the other notebooks.

In [1]:
import gym
import numpy as np
import pandas as pd
import json

## 01 Introduction to Reinforcement Learning

### Exercise 1

Finish implementing the `rollout_policy` function below, which should take an environment *and* a policy. Recall that the *policy* is a function that takes in a *state* and returns an *action*. The main difference is that instead of choosing a **random action**, like we just did (with poor results), the action should be chosen **with the policy** (as a function of the state).

In [2]:
env = gym.make('CartPole-v0')
print('Created env:', env)

Created env: <TimeLimit<CartPoleEnv<CartPole-v0>>>


In [3]:
def rollout_policy(env, policy):
    state = env.reset()
    
    done = False
    cumulative_reward = 0

    # Keep looping as long as the simulation has not finished.
    while not done:
        # Choose a random action (either 0 or 1).
        action = policy(state)
        
        # Take the action in the environment.
        state, reward, done, _ = env.step(action)
        
        # Update the cumulative reward.
        cumulative_reward += reward
        
    # Return the cumulative reward.
    return cumulative_reward

def sample_policy1(state):
    return 0 if state[0] < 0 else 1

def sample_policy2(state):
    return 1 if state[0] < 0 else 0

reward1 = np.mean([rollout_policy(env, sample_policy1) for _ in range(100)])
reward2 = np.mean([rollout_policy(env, sample_policy2) for _ in range(100)])

print('The first sample policy got an average reward of {}.'.format(reward1))
print('The second sample policy got an average reward of {}.'.format(reward2))

assert 5 < reward1 < 15, ('Make sure that rollout_policy computes the action '
                          'by applying the policy to the state.')
assert 25 < reward2 < 35, ('Make sure that rollout_policy computes the action '
                           'by applying the policy to the state.')

The first sample policy got an average reward of 9.44.
The second sample policy got an average reward of 28.76.


### Exercise 2

The current network and training configuration are too large and heavy-duty for a simple problem like CartPole. Modify the configuration to use a smaller network and to speed up the optimization of the surrogate objective. (Fewer SGD iterations and a larger batch size should help.)

In [4]:
import ray
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG
from ray.tune.logger import pretty_print

In [6]:
!../../tools/start-ray.sh --check --verbose

INFO: Ray is already running.


In [7]:
ray.init(address='auto', ignore_reinit_error=True, log_to_driver=False)

2020-06-13 10:48:06,679	WARNING worker.py:809 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.


{'node_ip_address': '192.168.1.149',
 'raylet_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:15832',
 'object_store_address': '/tmp/ray/session_2020-06-12_08-58-38_626987_40764/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-06-12_08-58-38_626987_40764/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-06-12_08-58-38_626987_40764'}

Here's one possible set. It takes longer for teh max reward to reach 200, so I increased the number of episodes `N` to 10.

In [8]:
config = DEFAULT_CONFIG.copy()
config['num_workers'] = 3
config['num_sgd_iter'] = 10                       # was 30
config['sgd_minibatch_size'] = 256                # was 128
config['model']['fcnet_hiddens'] = [20, 20]       # was [100, 100]
config['num_cpus_per_worker'] = 0

In [12]:
agent = PPOTrainer(config, 'CartPole-v0')

2020-06-13 10:49:44,314	INFO trainable.py:217 -- Getting current IP.
2020-06-13 10:49:44,315	WARNING util.py:37 -- Install gputil for GPU system monitoring.


In [13]:
N=20                # was 10
results = []
episode_data = []
episode_json = []
for n in range(N):
    result = agent.train()
    results.append(result)
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'],  
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']}    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    print(f'{n:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}')

  0: Min/Mean/Max reward:   8.0000/ 21.7088/110.0000
  1: Min/Mean/Max reward:   9.0000/ 25.0437/ 90.0000
  2: Min/Mean/Max reward:   8.0000/ 25.9161/ 78.0000
  3: Min/Mean/Max reward:  10.0000/ 27.0405/ 77.0000
  4: Min/Mean/Max reward:   9.0000/ 35.0885/101.0000
  5: Min/Mean/Max reward:  11.0000/ 39.2000/101.0000
  6: Min/Mean/Max reward:  13.0000/ 39.7212/ 85.0000
  7: Min/Mean/Max reward:  15.0000/ 46.6200/119.0000
  8: Min/Mean/Max reward:  14.0000/ 57.3800/158.0000
  9: Min/Mean/Max reward:   9.0000/ 62.8200/200.0000
 10: Min/Mean/Max reward:   9.0000/ 77.5000/200.0000
 11: Min/Mean/Max reward:  17.0000/ 91.6900/200.0000
 12: Min/Mean/Max reward:  17.0000/105.4800/200.0000
 13: Min/Mean/Max reward:  17.0000/121.6300/200.0000
 14: Min/Mean/Max reward:  19.0000/134.4900/200.0000
 15: Min/Mean/Max reward:  19.0000/142.3800/200.0000
 16: Min/Mean/Max reward:  19.0000/145.5600/200.0000
 17: Min/Mean/Max reward:  20.0000/154.1600/200.0000
 18: Min/Mean/Max reward:  28.0000/150.7400/20

In [14]:
df = pd.DataFrame(data=episode_data)
df

,n,episode_reward_min,episode_reward_mean,episode_reward_max,episode_len_mean
0,0,8.0,21.708791,110.0,21.708791
1,1,9.0,25.043750,90.0,25.043750
2,2,8.0,25.916129,78.0,25.916129
3,3,10.0,27.040541,77.0,27.040541
4,4,9.0,35.088496,101.0,35.088496
5,5,11.0,39.200000,101.0,39.200000
6,6,13.0,39.721154,85.0,39.721154
7,7,15.0,46.620000,119.0,46.620000
8,8,14.0,57.380000,158.0,57.380000
9,9,9.0,62.820000,200.0,62.820000


In [15]:
import sys
sys.path.append("../..")
from util.line_plots import plot_line, plot_line_with_min_max

import bokeh.io
# The next two lines prevent Bokeh from opening the graph in a new window.
bokeh.io.reset_output()
bokeh.io.output_notebook()

Loading BokehJS ...

In [16]:
plot_line_with_min_max(df, x_col='n', y_col='episode_reward_mean', min_col='episode_reward_min', max_col='episode_reward_max',
                      title='Episode Rewards', x_axis_label='n', y_axis_label='reward')

2020-06-13 11:30:59,513	ERROR import_thread.py:93 -- ImportThread: Connection closed by server.
2020-06-13 11:30:59,522	ERROR worker.py:1092 -- listen_error_messages_raylet: Connection closed by server.


([image](../../images/rllib/Cart-Pole-Episode-Rewards-Exercise.png))

Compare this graph with the graph in the lesson, where we used a stronger network:

![](../../images/rllib/Cart-Pole-Episode-Rewards.png)

Note that we only used 5 episodes before. If you compare the graphs at n=4, you see that this execise solution is training more slowly, but it after N=10, the mean reward grows quickly.

Try it again with slightly larger and/or small neural network layers.